In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 10)

##### Read in census data for 2012, 2016, and 2020

In [ ]:
df_race_2012 = pd.read_csv('data/election/census/census_race_2012.csv')
df_race_2016 = pd.read_csv('data/election/census/census_race_2016.csv')
df_race_2020 = pd.read_csv('data/election/census/census_race_2020.csv')

In [ ]:
df_age_sex_2012 = pd.read_csv('data/election/census/census_age_sex_2012.csv')
df_age_sex_2016 = pd.read_csv('data/election/census/census_age_sex_2016.csv')
df_age_sex_2020 = pd.read_csv('data/election/census/census_age_sex_2020.csv')

In [ ]:
df_age_employment_2012 = pd.read_csv('data/election/census/census_employment_2012.csv')
df_age_employment_2016 = pd.read_csv('data/election/census/census_employment_2016.csv')
df_age_employment_2020 = pd.read_csv('data/election/census/census_employment_2020.csv')

In [ ]:
df_industry_2012 = pd.read_csv('data/election/census/census_industry_2012.csv')
df_industry_2016 = pd.read_csv('data/election/census/census_industry_2016.csv')
df_industry_2020 = pd.read_csv('data/election/census/census_industry_2020.csv')

In [ ]:
df_housing_financial_2012 = pd.read_csv('data/election/census/census_housing_finanical_2012.csv')
df_housing_financial_2016 = pd.read_csv('data/election/census/census_housing_finanical_2016.csv')
df_housing_financial_2020 = pd.read_csv('data/election/census/census_housing_finanical_2020.csv')

In [ ]:
df_veteran_status_2012 = pd.read_csv('data/election/census/census_veteran_status_2012.csv')
df_veteran_status_2016 = pd.read_csv('data/election/census/census_veteran_status_2016.csv')
df_veteran_status_2020 = pd.read_csv('data/election/census/census_veteran_status_2020.csv')

##### Reshape race data

In [ ]:
import pandas as pd

def reshape_demographics_race_data(df):
    """
    Reshape demographic data from wide format to long format with separate county and state columns.
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with demographic data
    
    Returns:
    pandas.DataFrame: Reshaped dataframe with county and state as rows and demographic categories as columns
    """
    # Reset index to get the Label column as a regular column
    df = df.reset_index()
    
    # Melt the dataframe to convert counties from columns to rows
    melted = pd.melt(
        df,
        id_vars=['Label (Grouping)'],
        var_name='location',
        value_name='value'
    )
    
    # Split the location column into county and state
    # Remove the '!!Estimate' suffix and split on the last comma
    melted['location'] = melted['location'].str.replace('!!Estimate', '')
    melted[['county_name', 'state_name']] = melted['location'].str.rsplit(',', n=1, expand=True)
    
    # Strip whitespace from county and state names
    melted['county_name'] = melted['county_name'].str.strip()
    melted['state_name'] = melted['state_name'].str.strip()
    
    # Pivot the data to get demographic categories as columns
    reshaped = melted.pivot(
        index=['county_name', 'state_name'],
        columns='Label (Grouping)',
        values='value'
    ).reset_index()
    
    # Clean up column names by removing any extra levels
    reshaped.columns.name = None
    
    # Reorder columns to have county_name and state_name first
    demographic_cols = [col for col in reshaped.columns if col not in ['county_name', 'state_name']]
    column_order = ['county_name', 'state_name'] + demographic_cols
    
    return reshaped[column_order]

In [ ]:
df_race_2012_reshaped = reshape_demographics_race_data(df_race_2012)
df_race_2012_reshaped = df_race_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_race_2016_reshaped = reshape_demographics_race_data(df_race_2016)
df_race_2016_reshaped = df_race_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_race_2020_reshaped = reshape_demographics_race_data(df_race_2020)
df_race_2020_reshaped = df_race_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_race_2012_reshaped.to_csv('data/election/census/census_race_2012_reshaped.csv', index=False)
df_race_2016_reshaped.to_csv('data/election/census/census_race_2016_reshaped.csv', index=False)
df_race_2020_reshaped.to_csv('data/election/census/census_race_2020_reshaped.csv', index=False)

##### Reshape age data

In [ ]:
import pandas as pd
import numpy as np

def reshape_demographics_age_data(df):
    # First, let's get the column names and split them into components
    columns = df.columns[1:]  # Skip the 'Label (Grouping)' column
    
    # Create a list to store the transformed data
    transformed_data = []
    
    # Track unique counties and states
    processed_locations = set()
    
    # Iterate through each location column
    for col in columns:
        # Split the column name
        parts = col.split('!!')
        
        # Extract location and demographic type
        location = parts[0]
        demo_type = parts[1]  # Male, Female, or Total
        estimate = parts[2] if len(parts) > 2 else None
        
        # Skip if not an Estimate
        if estimate != 'Estimate':
            continue
            
        # Split location into county and state
        county_state = location.split(', ')
        county = county_state[0]
        state = county_state[1]
        
        # Create unique key for this county
        location_key = (county, state)
        
        # If we haven't processed this county yet, create a new row
        if location_key not in processed_locations:
            row_data = {
                'county_name': county,
                'state_name': state
            }
            
            # Add data for each demographic label
            for idx, label in enumerate(df['Label (Grouping)']):
                if pd.notna(df.iloc[idx][col]) and str(df.iloc[idx][col]) != '(X)':
                    value = df.iloc[idx][col]
                    # Convert percentages to floats
                    if isinstance(value, str) and '%' in value:
                        value = float(value.strip('%'))/100
                    
                    # Create column name combining label and demographic type
                    column_name = f"{label}_{demo_type.lower()}"
                    row_data[column_name] = value
            
            transformed_data.append(row_data)
            processed_locations.add(location_key)
        else:
            # Update existing row with additional demographic type data
            row_idx = next(i for i, row in enumerate(transformed_data) 
                          if row['county_name'] == county and row['state_name'] == state)
            
            for idx, label in enumerate(df['Label (Grouping)']):
                if pd.notna(df.iloc[idx][col]) and str(df.iloc[idx][col]) != '(X)':
                    value = df.iloc[idx][col]
                    # Convert percentages to floats
                    if isinstance(value, str) and '%' in value:
                        value = float(value.strip('%'))/100
                    
                    # Create column name combining label and demographic type
                    column_name = f"{label}_{demo_type.lower()}"
                    transformed_data[row_idx][column_name] = value
    
    # Create new dataframe
    new_df = pd.DataFrame(transformed_data)
    
    # Sort columns alphabetically after county_name and state_name
    cols = ['county_name', 'state_name'] + sorted([col for col in new_df.columns 
                                                 if col not in ['county_name', 'state_name']])
    new_df = new_df[cols]
    
    return new_df

# Example usage:
# reshaped_df = reshape_demographics_data(your_df)

In [ ]:
df_age_sex_2012_reshaped = reshape_demographics_age_data(df_age_sex_2012)
df_age_sex_2012_reshaped = df_age_sex_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_age_sex_2016_reshaped = reshape_demographics_age_data(df_age_sex_2016)
df_age_sex_2016_reshaped = df_age_sex_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_age_sex_2020_reshaped = reshape_demographics_age_data(df_age_sex_2020)
df_age_sex_2020_reshaped = df_age_sex_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_age_sex_2012_reshaped.to_csv('data/election/census/census_age_sex_2012_reshaped.csv', index=False)
df_age_sex_2016_reshaped.to_csv('data/election/census/census_age_sex_2016_reshaped.csv', index=False)
df_age_sex_2020_reshaped.to_csv('data/election/census/census_age_sex_2020_reshaped.csv', index=False)

##### Reshape Employment data

In [ ]:
import pandas as pd
import numpy as np

def reshape_employment_data(df, year):
    """
    Reshape county-level census data from wide format to long format with counties as rows,
    creating separate columns for Total, Labor Force, Employed, and Unemployment rate for each category.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with county data in wide format
    year (int): Census year (2012 or 2016/2020) to determine column naming format
    
    Returns:
    pandas.DataFrame: Reshaped DataFrame with counties as rows
    """
    # Define metric mappings based on year
    if year == 2012:
        metric_mappings = {
            'Total': 'Total',
            'In labor force': 'labor_force',
            'Employed': 'employed',
            'Unemployment rate': 'unemployment_rate'
        }
    else:  # 2016 or 2020
        metric_mappings = {
            'Total': 'Total',
            'Labor Force Participation Rate': 'labor_force',
            'Employment/Population Ratio': 'employed',
            'Unemployment rate': 'unemployment_rate'
        }
    
    # Get the Label column values as our new column names, filtering out header rows
    row_categories = []
    for idx, label in enumerate(df['Label (Grouping)']):
        if pd.notna(label):
            # Skip if row is all caps and contains no data (header row)
            if not (str(label).isupper() and df.iloc[idx, 1:].isna().all()):
                row_categories.append((idx, label))
    
    # Process each unique county
    counties_data = []
    
    # Get unique county-state combinations
    county_columns = [col for col in df.columns if '!!' in col]
    unique_counties = set()
    for col in county_columns:
        county_state = col.split('!!')[0]
        if ',' in county_state:
            unique_counties.add(county_state)
            
    # Process each county
    for county_state in unique_counties:
        county_name, state_name = county_state.split(',', 1)
        state_name = state_name.strip()
        
        # Create a dictionary for this county's data
        county_data = {
            'county_name': county_name,
            'state_name': state_name
        }
        print('processing', county_name, state_name)
        
        # For each demographic category
        for idx, category in row_categories:
            category_clean = (category.replace(' ', '_')
                            .replace('(', '')
                            .replace(')', '')
                            .replace(',', '')
                            .replace('/', '_')
                            .replace('-', '_')
                            .replace('...', ''))
            
            # Get the four metrics for this category using the appropriate mapping
            for original_metric, clean_metric in metric_mappings.items():
                col_name = f"{county_state}!!{original_metric}!!Estimate"
                if col_name in df.columns:
                    value = df.iloc[idx][col_name]
                    
                    # Convert percentage strings to floats if possible
                    if isinstance(value, str):
                        if value in ['-', '(X)']:
                            value = None
                        elif '%' in value:
                            try:
                                value = float(value.strip('%')) / 100
                            except ValueError:
                                pass
                    
                    # Create clean column name
                    final_col_name = f"{category_clean}_{clean_metric}"
                    county_data[final_col_name] = value
        
        counties_data.append(county_data)
    
    # Create the final DataFrame
    result_df = pd.DataFrame(counties_data)
    
    return result_df

In [ ]:
df_age_employment_2012_reshaped = reshape_employment_data(df_age_employment_2012, 2012)
df_age_employment_2012_reshaped = df_age_employment_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
df_age_employment_2016_reshaped = reshape_employment_data(df_age_employment_2016, 2016)
df_age_employment_2016_reshaped = df_age_employment_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
df_age_employment_2020_reshaped = reshape_employment_data(df_age_employment_2020, 2020)
df_age_employment_2020_reshaped = df_age_employment_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_age_employment_2012_reshaped.to_csv('data/election/census/census_age_employment_2012_reshaped.csv', index=False)
df_age_employment_2016_reshaped.to_csv('data/election/census/census_age_employment_2016_reshaped.csv', index=False)
df_age_employment_2020_reshaped.to_csv('data/election/census/census_age_employment_2020_reshaped.csv', index=False)

##### Reshape Industry data

In [ ]:
import pandas as pd
import numpy as np

def reshape_industry_data(df):
    """
    Reshape county-level occupation data from wide format to long format with counties as rows,
    creating separate columns for total workers and percentage in each occupation category.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with county occupation data in wide format
    
    Returns:
    pandas.DataFrame: Reshaped DataFrame with counties as rows
    """
    # Get the Label column values as our new column names, filtering out header rows
    row_categories = []
    for idx, label in enumerate(df['Label (Grouping)']):
        if pd.notna(label):
            # Skip if row is all caps or percent imputed rows
            if not (str(label).isupper() or 'PERCENT' in str(label)):
                row_categories.append((idx, label))
    
    # Define occupation categories in order
    occupation_categories = [
        'Total',
        'Management, business, science, and arts occupations',
        'Service occupations',
        'Sales and office occupations',
        'Natural resources, construction, and maintenance occupations',
        'Production, transportation, and material moving occupations'
    ]
    
    # Process each unique county
    counties_data = []
    
    # Get unique county-state combinations
    county_columns = [col for col in df.columns if '!!' in col]
    unique_counties = set()
    for col in county_columns:
        county_state = col.split('!!')[0]
        if ',' in county_state:
            unique_counties.add(county_state)
    
    # Process each county
    for county_state in unique_counties:
        county_name, state_name = county_state.split(',', 1)
        state_name = state_name.strip()
        
        # Create a dictionary for this county's data
        county_data = {
            'county_name': county_name,
            'state_name': state_name
        }
        
        # For each industry category
        for idx, category in row_categories:
            category_clean = (category.replace(' ', '_')
                            .replace('(', '')
                            .replace(')', '')
                            .replace(',', '')
                            .replace('/', '_')
                            .replace('-', '_')
                            .replace('.', '')
                            .replace('...', ''))
            
            # Get values for total and each occupation percentage
            for occupation in occupation_categories:
                col_name = f"{county_state}!!{occupation}!!Estimate"
                if col_name in df.columns:
                    value = df.iloc[idx][col_name]
                    
                    # Convert percentage strings to floats if possible
                    if isinstance(value, str):
                        if value == '-' or value == '(X)':
                            value = None
                        elif '%' in value:
                            try:
                                value = float(value.strip('%')) / 100
                            except ValueError:
                                pass
                    
                    # Create clean column name
                    if occupation == 'Total':
                        suffix = 'total_workers'
                    else:
                        occupation_clean = (occupation.replace(' ', '_')
                                         .replace(',', '')
                                         .replace('_and_', '_')
                                         .lower())
                        suffix = f'pct_{occupation_clean}'
                    
                    final_col_name = f"{category_clean}_{suffix}"
                    county_data[final_col_name] = value
        
        counties_data.append(county_data)
    
    # Create the final DataFrame
    result_df = pd.DataFrame(counties_data)
    
    return result_df

In [ ]:
df_industry_2012_reshaped = reshape_industry_data(df_industry_2012)
df_industry_2012_reshaped = df_industry_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_industry_2016_reshaped = reshape_industry_data(df_industry_2016)
df_industry_2016_reshaped = df_industry_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_industry_2020_reshaped = reshape_industry_data(df_industry_2020)
df_industry_2020_reshaped = df_industry_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_industry_2012_reshaped.to_csv('data/election/census/census_industry_2012_reshaped.csv', index=False)
df_industry_2016_reshaped.to_csv('data/election/census/census_industry_2016_reshaped.csv', index=False)
df_industry_2020_reshaped.to_csv('data/election/census/census_industry_2020_reshaped.csv', index=False)

##### Reshape housing financial data

In [ ]:
import pandas as pd
import numpy as np

def reshape_housing_financial_data(df):
    """
    Reshape county-level housing financial data from wide format to long format with counties as rows,
    creating separate columns for total, owner-occupied, and renter-occupied housing metrics.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with county housing data in wide format
    
    Returns:
    pandas.DataFrame: Reshaped DataFrame with counties as rows
    """
    # Get the Label column values as our new column names, filtering out header rows
    row_categories = []
    for idx, label in enumerate(df['Label (Grouping)']):
        if pd.notna(label):
            # Skip if row is all caps, header rows, or percent imputed
            if not (str(label).isupper() or 'PERCENT' in str(label)):
                row_categories.append((idx, label))
    
    # Define housing categories
    housing_categories = [
        'Occupied housing units',
        'Owner-occupied housing units',
        'Renter-occupied housing units'
    ]
    
    # Process each unique county
    counties_data = []
    
    # Get unique county-state combinations
    county_columns = [col for col in df.columns if '!!' in col]
    unique_counties = set()
    for col in county_columns:
        county_state = col.split('!!')[0]
        if ',' in county_state:
            unique_counties.add(county_state)
    
    # Process each county
    for county_state in unique_counties:
        county_name, state_name = county_state.split(',', 1)
        state_name = state_name.strip()
        
        # Create a dictionary for this county's data
        county_data = {
            'county_name': county_name,
            'state_name': state_name
        }
        
        # For each financial metric
        for idx, category in row_categories:
            category_clean = (category.replace(' ', '_')
                            .replace('(', '')
                            .replace(')', '')
                            .replace(',', '')
                            .replace('/', '_')
                            .replace('-', '_')
                            .replace('.', '')
                            .replace('...', '')
                            .replace('$', '')
                            .replace('__', '_')
                            .lower())
            
            # Get values for each housing category
            for housing_type in housing_categories:
                col_name = f"{county_state}!!{housing_type}!!Estimate"
                if col_name in df.columns:
                    value = df.iloc[idx][col_name]
                    
                    # Convert percentage strings to floats if possible
                    if isinstance(value, str):
                        if value == '-' or value == '(X)':
                            value = None
                        elif '%' in value:
                            try:
                                value = float(value.strip('%')) / 100
                            except ValueError:
                                pass
                        # Handle dollar amounts
                        elif '$' in value or ',' in value:
                            try:
                                value = float(value.replace('$', '').replace(',', ''))
                            except ValueError:
                                pass
                    
                    # Create clean column name
                    housing_type_clean = (housing_type.replace(' ', '_')
                                        .replace('-', '_')
                                        .lower())
                    final_col_name = f"{category_clean}_{housing_type_clean}"
                    county_data[final_col_name] = value
        
        counties_data.append(county_data)
    
    # Create the final DataFrame
    result_df = pd.DataFrame(counties_data)
    
    return result_df

In [ ]:
df_housing_financial_2012_reshaped = reshape_housing_financial_data(df_housing_financial_2012)
df_housing_financial_2012_reshaped = df_housing_financial_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_housing_financial_2016_reshaped = reshape_housing_financial_data(df_housing_financial_2016)
df_housing_financial_2016_reshaped = df_housing_financial_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_housing_financial_2020_reshaped = reshape_housing_financial_data(df_housing_financial_2020)
df_housing_financial_2020_reshaped = df_housing_financial_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_housing_financial_2012_reshaped.to_csv('data/election/census/census_housing_financial_2012_reshaped.csv', index=False)
df_housing_financial_2016_reshaped.to_csv('data/election/census/census_housing_financial_2016_reshaped.csv', index=False)
df_housing_financial_2020_reshaped.to_csv('data/election/census/census_housing_financial_2020_reshaped.csv', index=False)

##### Reshape veteran status data

In [ ]:
import pandas as pd
import numpy as np

def reshape_veteran_data(df):
    """
    Reshape county-level veteran status data from wide format to long format with counties as rows,
    creating separate columns for total population, veterans, and nonveterans metrics.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with county veteran data in wide format
    
    Returns:
    pandas.DataFrame: Reshaped DataFrame with counties as rows
    """
    # Get the Label column values as our new column names, filtering out header rows
    row_categories = []
    for idx, label in enumerate(df['Label (Grouping)']):
        if pd.notna(label):
            # Skip if row is all caps, header rows, or percent imputed
            if not (str(label).isupper() or 'PERCENT' in str(label)):
                row_categories.append((idx, label))
    
    # Define status categories
    status_categories = [
        'Total',
        'Veterans',
        'Nonveterans'
    ]
    
    # Process each unique county
    counties_data = []
    
    # Get unique county-state combinations
    county_columns = [col for col in df.columns if '!!' in col]
    unique_counties = set()
    for col in county_columns:
        county_state = col.split('!!')[0]
        if ',' in county_state:
            unique_counties.add(county_state)
    
    # Process each county
    for county_state in unique_counties:
        county_name, state_name = county_state.split(',', 1)
        state_name = state_name.strip()
        
        # Create a dictionary for this county's data
        county_data = {
            'county_name': county_name,
            'state_name': state_name
        }
        
        # For each demographic/service metric
        for idx, category in row_categories:
            category_clean = (category.replace(' ', '_')
                            .replace('(', '')
                            .replace(')', '')
                            .replace(',', '')
                            .replace('/', '_')
                            .replace('-', '_')
                            .replace('.', '')
                            .replace('...', '')
                            .replace('$', '')
                            .replace('__', '_')
                            .lower())
            
            # Special handling for period of service categories
            is_period_of_service = 'veterans' in category_clean and any(period in category_clean 
                                                                      for period in ['gulf_war', 'vietnam', 'korean', 'world_war'])
            
            # Get values for each status category
            for status in status_categories:
                col_name = f"{county_state}!!{status}!!Estimate"
                if col_name in df.columns:
                    value = df.iloc[idx][col_name]
                    
                    # Convert percentage strings to floats if possible
                    if isinstance(value, str):
                        if value == '-' or value == '(X)':
                            value = None
                        elif '%' in value:
                            try:
                                value = float(value.strip('%')) / 100
                            except ValueError:
                                pass
                        # Handle dollar amounts
                        elif '$' in value or ',' in value:
                            try:
                                value = float(value.replace('$', '').replace(',', ''))
                            except ValueError:
                                pass
                    
                    # Create clean column name
                    if is_period_of_service:
                        # For period of service, only store veteran percentage
                        if status == 'Veterans':
                            final_col_name = f"pct_{category_clean}"
                            county_data[final_col_name] = value
                    else:
                        # For other metrics, store all three categories
                        status_clean = status.lower()
                        final_col_name = f"{category_clean}_{status_clean}"
                        county_data[final_col_name] = value
        
        counties_data.append(county_data)
    
    # Create the final DataFrame
    result_df = pd.DataFrame(counties_data)
    
    return result_df

In [ ]:
df_veteran_status_2012_reshaped = reshape_veteran_data(df_veteran_status_2012)
df_veteran_status_2012_reshaped = df_veteran_status_2012_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_veteran_status_2016_reshaped = reshape_veteran_data(df_veteran_status_2016)
df_veteran_status_2016_reshaped = df_veteran_status_2016_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)
df_veteran_status_2020_reshaped = reshape_veteran_data(df_veteran_status_2020)
df_veteran_status_2020_reshaped = df_veteran_status_2020_reshaped.sort_values(by=['state_name', 'county_name']).reset_index(drop=True)

In [ ]:
# save the reshaped data to csvs
df_veteran_status_2012_reshaped.to_csv('data/election/census/census_veteran_status_2012_reshaped.csv', index=False)
df_veteran_status_2016_reshaped.to_csv('data/election/census/census_veteran_status_2016_reshaped.csv', index=False)
df_veteran_status_2020_reshaped.to_csv('data/election/census/census_veteran_status_2020_reshaped.csv', index=False)

##### Stack dataframes in list for processing

In [ ]:
# race dataframes
df_race_list = [df_race_2012_reshaped, df_race_2016_reshaped, df_race_2020_reshaped]
# age_sex dataframes
df_age_sex_list = [df_age_sex_2012_reshaped, df_age_sex_2016_reshaped, df_age_sex_2020_reshaped]
# age_employment dataframes
df_age_employment_list = [df_age_employment_2012_reshaped, df_age_employment_2016_reshaped, df_age_employment_2020_reshaped]
# industry dataframes
df_industry_list = [df_industry_2012_reshaped, df_industry_2016_reshaped, df_industry_2020_reshaped]
# housing_financial dataframes
df_housing_financial_list = [df_housing_financial_2012_reshaped, df_housing_financial_2016_reshaped, df_housing_financial_2020_reshaped]
# veteran_status dataframes
df_veteran_status_list = [df_veteran_status_2012_reshaped, df_veteran_status_2016_reshaped, df_veteran_status_2020_reshaped]

##### Fix data

In [ ]:
# drop the row with county_name 'index' in the df_race_list dataframes
for df in df_race_list:
    df.drop(df[df['county_name'] == 'index'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

In [ ]:
# drop rows where state_name is 'Puerto Rico' or 'Alaska' or 'Hawaii' or 'District of Columbia' in all dataframe lists
for df in df_race_list:
    df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
    df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

for df in df_age_sex_list:
    df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
    df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

for df in df_age_employment_list:
    df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
    df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

for df in df_industry_list:
    df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
    df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

for df in df_housing_financial_list:
    df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
    df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

for df in df_veteran_status_list:
    df.drop(df[df['state_name'] == 'Puerto Rico'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Alaska'].index, inplace=True)
    df.drop(df[df['state_name'] == 'Hawaii'].index, inplace=True)
    df.drop(df[df['state_name'] == 'District of Columbia'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

In [ ]:
# show rows where county_name does not contain 'County' or 'Parish' or 'city' or 'Borough
df2[~df2['county_name'].str.contains('County|Parish|city|Borough')]

In [ ]:
state_abbr_to_name = {
    'AL': 'Alabama',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

state_name_to_abbr = {v: k for k, v in state_abbr_to_name.items()}

In [ ]:
# df2['state_abbr'] = df2['state_name'].map(state_name_to_abbr)

# for all dataframes lists, add a column for state_abbr
for df in df_race_list:
    df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

for df in df_age_sex_list:
    df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

for df in df_age_employment_list:
    df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

for df in df_industry_list:
    df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

for df in df_housing_financial_list:
    df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

for df in df_veteran_status_list:
    df['state_abbr'] = df['state_name'].map(state_name_to_abbr)

##### Add FIPS codes

In [ ]:
import pandas as pd
import unidecode

def clean_place_name(name):
    """Clean place names for consistent matching"""
    if pd.isna(name):
        return name
    
    # Convert to string and clean accented characters
    name = str(name)
    name = unidecode.unidecode(name)
    
    # Standardize spaces and case
    name = name.strip()
    
    # Standardize specific terms
    replacements = {
        'Municipality': 'Municipio',
        'City and Borough': 'Borough',
        'Census Area': 'Census Area',
        ' city': ' City',
        'LaSalle': 'La Salle',
        'LaPorte': 'La Porte',
        'DeSoto': 'De Soto',
        'DeKalb': 'De Kalb',
        'St.': 'St',
        'Ste.': 'Ste'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    return name

def add_fips_codes(census_df, fips_csv_path):
    """
    Add FIPS codes to census data by matching county and state.
    Handles special cases for Alaska, Puerto Rico, and Virginia.
    
    Parameters:
    census_df (pandas.DataFrame): Census dataframe with county_name and state_abbr columns
    fips_csv_path (str): Path to the FIPS CSV file
    
    Returns:
    pandas.DataFrame: Census dataframe with FIPS codes added
    """
    # Read FIPS data
    fips_df = pd.read_csv(fips_csv_path)
    
    # Filter out non-place rows (like "UNITED STATES" and state-level entries)
    fips_df = fips_df[~fips_df['name'].isna()]
    fips_df = fips_df[fips_df['name'].str.contains('County|Borough|Parish|Census Area|city|Municipio', 
                                                  case=False, na=False)]
    
    # Clean up names in both dataframes
    census_df = census_df.copy()
    census_df['clean_name'] = census_df['county_name'].apply(clean_place_name)
    fips_df['clean_name'] = fips_df['name'].apply(clean_place_name)
    
    # # Special handling for DC
    # dc_mask = census_df['county_name'] == 'District of Columbia'
    # census_df.loc[dc_mask, 'state_abbr'] = 'DC'
    
    # # Handle cases where state_abbr might be NaN
    # state_map = {
    #     'Borough': 'AK',
    #     'Census Area': 'AK',
    #     'Municipio': 'PR'
    # }
    
    # for identifier, state_code in state_map.items():
    #     mask = census_df['county_name'].str.contains(identifier, na=False)
    #     census_df.loc[mask, 'state_abbr'] = state_code
    
    # Merge the dataframes
    result = pd.merge(
        census_df,
        fips_df[['fips', 'clean_name', 'state']],
        how='left',
        left_on=['clean_name', 'state_abbr'],
        right_on=['clean_name', 'state']
    )
    
    # Drop the working columns
    result = result.drop(['clean_name', 'state'], axis=1)
    
    # Convert FIPS codes to strings with leading zeros
    result['fips'] = result['fips'].astype(str).str.zfill(5)
    
    # Rename the FIPS column
    result = result.rename(columns={'fips': 'fips_code'})
    
    # Check for unmatched counties
    unmatched = result[result['fips_code'].isna()]
    if len(unmatched) > 0:
        print(f"\nWarning: {len(unmatched)} places could not be matched with FIPS codes:")
        for _, row in unmatched.iterrows():
            print(f"- {row['county_name']}, {row['state_abbr']}")
    
    # Reorder columns to put fips_code after state_name
    cols = list(result.columns)
    state_name_idx = cols.index('state_name')
    cols.remove('fips_code')
    cols.insert(state_name_idx + 1, 'fips_code')
    result = result[cols]
    
    return result

In [ ]:
# # add fips codes to all dataframes
# for df in df_race_list:
#     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# for df in df_age_sex_list:
#     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# for df in df_age_employment_list:
#     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# for df in df_industry_list:
#     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# for df in df_housing_financial_list:
#     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

# for df in df_veteran_status_list:
#     df = add_fips_codes(df, 'data/election/fips_state_and_county.csv')

df_race_2012_reshaped = add_fips_codes(df_race_2012_reshaped, 'data/election/fips_state_and_county.csv')
df_race_2016_reshaped = add_fips_codes(df_race_2016_reshaped, 'data/election/fips_state_and_county.csv')
df_race_2020_reshaped = add_fips_codes(df_race_2020_reshaped, 'data/election/fips_state_and_county.csv')
df_age_sex_2012_reshaped = add_fips_codes(df_age_sex_2012_reshaped, 'data/election/fips_state_and_county.csv')
df_age_sex_2016_reshaped = add_fips_codes(df_age_sex_2016_reshaped, 'data/election/fips_state_and_county.csv')
df_age_sex_2020_reshaped = add_fips_codes(df_age_sex_2020_reshaped, 'data/election/fips_state_and_county.csv')
df_age_employment_2012_reshaped = add_fips_codes(df_age_employment_2012_reshaped, 'data/election/fips_state_and_county.csv')
df_age_employment_2016_reshaped = add_fips_codes(df_age_employment_2016_reshaped, 'data/election/fips_state_and_county.csv')
df_age_employment_2020_reshaped = add_fips_codes(df_age_employment_2020_reshaped, 'data/election/fips_state_and_county.csv')
df_industry_2012_reshaped = add_fips_codes(df_industry_2012_reshaped, 'data/election/fips_state_and_county.csv')
df_industry_2016_reshaped = add_fips_codes(df_industry_2016_reshaped, 'data/election/fips_state_and_county.csv')
df_industry_2020_reshaped = add_fips_codes(df_industry_2020_reshaped, 'data/election/fips_state_and_county.csv')
df_housing_financial_2012_reshaped = add_fips_codes(df_housing_financial_2012_reshaped, 'data/election/fips_state_and_county.csv')
df_housing_financial_2016_reshaped = add_fips_codes(df_housing_financial_2016_reshaped, 'data/election/fips_state_and_county.csv')
df_housing_financial_2020_reshaped = add_fips_codes(df_housing_financial_2020_reshaped, 'data/election/fips_state_and_county.csv')
df_veteran_status_2012_reshaped = add_fips_codes(df_veteran_status_2012_reshaped, 'data/election/fips_state_and_county.csv')
df_veteran_status_2016_reshaped = add_fips_codes(df_veteran_status_2016_reshaped, 'data/election/fips_state_and_county.csv')
df_veteran_status_2020_reshaped = add_fips_codes(df_veteran_status_2020_reshaped, 'data/election/fips_state_and_county.csv')


In [ ]:
pd.set_option('display.max_rows', 200)

In [125]:
# drop all columns with more than 0 missing values
df_age_employment_2012_reshaped.dropna(axis=1, how='any', inplace=True)
df_age_employment_2016_reshaped.dropna(axis=1, how='any', inplace=True)
df_age_employment_2020_reshaped.dropna(axis=1, how='any', inplace=True)
df_industry_2012_reshaped.dropna(axis=1, how='any', inplace=True)
df_industry_2016_reshaped.dropna(axis=1, how='any', inplace=True)
df_industry_2020_reshaped.dropna(axis=1, how='any', inplace=True)
df_housing_financial_2012_reshaped.dropna(axis=1, how='any', inplace=True)
df_housing_financial_2016_reshaped.dropna(axis=1, how='any', inplace=True)
df_housing_financial_2020_reshaped.dropna(axis=1, how='any', inplace=True)
df_veteran_status_2012_reshaped.dropna(axis=1, how='any', inplace=True)
df_veteran_status_2016_reshaped.dropna(axis=1, how='any', inplace=True)
df_veteran_status_2020_reshaped.dropna(axis=1, how='any', inplace=True)

In [ ]:
# add _2012 to all columns except for county_name, state_name, fips_code, state_abbr
df_race_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_race_2012_reshaped.columns]
df_age_sex_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_sex_2012_reshaped.columns]
df_age_employment_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_employment_2012_reshaped.columns]
df_industry_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_industry_2012_reshaped.columns]
df_housing_financial_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_housing_financial_2012_reshaped.columns]
df_veteran_status_2012_reshaped.columns = [col + '_2012' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_veteran_status_2012_reshaped.columns]

# add _2016 to all columns except for county_name, state_name, fips_code, state_abbr
df_race_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_race_2016_reshaped.columns]
df_age_sex_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_sex_2016_reshaped.columns]
df_age_employment_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_employment_2016_reshaped.columns]
df_industry_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_industry_2016_reshaped.columns]
df_housing_financial_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_housing_financial_2016_reshaped.columns]
df_veteran_status_2016_reshaped.columns = [col + '_2016' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_veteran_status_2016_reshaped.columns]

# add _2020 to all columns except for county_name, state_name, fips_code, state_abbr
df_race_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_race_2020_reshaped.columns]
df_age_sex_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_sex_2020_reshaped.columns]
df_age_employment_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_age_employment_2020_reshaped.columns]
df_industry_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_industry_2020_reshaped.columns]
df_housing_financial_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_housing_financial_2020_reshaped.columns]
df_veteran_status_2020_reshaped.columns = [col + '_2020' if col not in ['county_name', 'state_name', 'fips_code', 'state_abbr'] else col for col in df_veteran_status_2020_reshaped.columns]

In [202]:
df_race_2020_reshaped

,county_name,state_name,fips_code,Total:_2020,American Indian and Alaska Native alone_2020,...,Two or more races:_2020,White alone_2020,"Two races excluding Some other race, and three or more races_2020",Two races including Some other race_2020,state_abbr
0,Autauga County,Alabama,1001.0,"55,639",155,...,"1,421","42,150","1,075",346,AL
1,Baldwin County,Alabama,1003.0,"218,289","1,514",...,"5,677","186,504","4,158","1,519",AL
2,Barbour County,Alabama,1005.0,"25,026",88,...,522,"11,587",290,232,AL
3,Bibb County,Alabama,1007.0,"22,374",12,...,114,"17,138",114,0,AL
4,Blount County,Alabama,1009.0,"57,755",55,...,"1,293","54,271","1,029",264,AL
...,...,...,...,...,...,...,...,...,...,...,...
3102,Sweetwater County,Wyoming,56037.0,"43,352",477,...,"1,844","39,470",957,887,WY
3103,Teton County,Wyoming,56039.0,"23,356",38,...,432,"20,815",237,195,WY
3104,Uinta County,Wyoming,56041.0,"20,374",122,...,867,"18,929",760,107,WY
3105,Washakie County,Wyoming,56043.0,"7,933",69,...,376,"6,898",308,68,WY


In [ ]:
# merge all dataframes on fips_code
df_race_merged = df_race_2012_reshaped.merge(df_race_2016_reshaped, on='fips_code')
df_race_merged = df_race_merged.merge(df_race_2020_reshaped, on='fips_code')
df_age_sex_merged = df_age_sex_2012_reshaped.merge(df_age_sex_2016_reshaped, on='fips_code')
df_age_sex_merged = df_age_sex_merged.merge(df_age_sex_2020_reshaped, on='fips_code')
df_age_employment_merged = df_age_employment_2012_reshaped.merge(df_age_employment_2016_reshaped, on='fips_code')
df_age_employment_merged = df_age_employment_merged.merge(df_age_employment_2020_reshaped, on='fips_code')
df_industry_merged = df_industry_2012_reshaped.merge(df_industry_2016_reshaped, on='fips_code')
df_industry_merged = df_industry_merged.merge(df_industry_2020_reshaped, on='fips_code')
df_housing_financial_merged = df_housing_financial_2012_reshaped.merge(df_housing_financial_2016_reshaped, on='fips_code')
df_housing_financial_merged = df_housing_financial_merged.merge(df_housing_financial_2020_reshaped, on='fips_code')
df_veteran_status_merged = df_veteran_status_2012_reshaped.merge(df_veteran_status_2016_reshaped, on='fips_code')
df_veteran_status_merged = df_veteran_status_merged.merge(df_veteran_status_2020_reshaped, on='fips_code')

In [142]:
import geopandas as gpd

# load the existing counties data county_data_with_elections_2012_2016_2020.geojson
counties = gpd.read_file('data/election/final_data/county_data_with_elections_2012_2016_2020.geojson')

In [146]:
df_veteran_status_merged['fips_code']

0        1001.0
1        1003.0
2        1005.0
3        1007.0
4        1009.0
         ...   
3102    56037.0
3103    56039.0
3104    56041.0
3105    56043.0
3106    56045.0
Name: fips_code, Length: 3107, dtype: object

In [145]:
counties['FIPS Code']

0        1001
1        1003
2        1005
3        1007
4        1009
        ...  
3101    56037
3102    56039
3103    56041
3104    56043
3105    56045
Name: FIPS Code, Length: 3106, dtype: object

In [149]:
# add a .0 to each fips code in the counties dataframe
counties['FIPS Code'] = counties['FIPS Code'].astype(str) + '.0'

In [152]:
# merging all data to the counties dataframe
counties_merged = counties.merge(df_race_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_age_sex_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_age_employment_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_industry_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_housing_financial_merged, left_on='FIPS Code', right_on='fips_code')
counties_merged = counties_merged.merge(df_veteran_status_merged, left_on='FIPS Code', right_on='fips_code')

C:\Users\bovam\AppData\Local\Temp\ipykernel_21892\3259534277.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'state_name_x_x', 'fips_code_x', 'state_name_y_x', 'county_name_x_x', 'county_name_y_x', 'state_abbr_y_x', 'state_abbr_x_x'} in the result is deprecated and will raise a MergeError in a future version.
  counties_merged = counties_merged.merge(df_industry_merged, left_on='FIPS Code', right_on='fips_code')
C:\Users\bovam\AppData\Local\Temp\ipykernel_21892\3259534277.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'state_name_x_x', 'fips_code_x', 'state_name_y_x', 'county_name_x_x', 'county_name_y_x', 'state_abbr_y_x', 'state_abbr_x_x'} in the result is deprecated and will raise a MergeError in a future version.
  counties_merged = counties_merged.merge(df_veteran_status_merged, left_on='FIPS Code', right_on='fips_code')


In [153]:
counties_merged

,state_abbr,total_votes_2012,votes_dem_2012,votes_gop_2012,per_dem_2012,...,with_any_disability_nonveterans_2016,without_a_disability_total_2016,without_a_disability_veterans_2016,without_a_disability_nonveterans_2016,state_abbr_y_y
0,AL,23909.0,6354.0,17366.0,0.265758,...,7316.0,31732.0,3520.0,28212.0,AL
1,AL,84988.0,18329.0,65772.0,0.215666,...,20021.0,126655.0,13079.0,113576.0,AL
2,AL,11459.0,5873.0,5539.0,0.512523,...,3982.0,13296.0,869,12427.0,AL
3,AL,8391.0,2200.0,6131.0,0.262186,...,2486.0,13023.0,821,12202.0,AL
4,AL,23980.0,2961.0,20741.0,0.123478,...,6603.0,35452.0,2847.0,32605.0,AL
...,...,...,...,...,...,...,...,...,...,...,...
3100,WY,16750.0,4773.0,11427.0,0.284955,...,4002.0,27120.0,2039.0,25081.0,WY
3101,WY,11356.0,6211.0,4858.0,0.546936,...,1047.0,16886.0,1089.0,15797.0,WY
3102,WY,8453.0,1628.0,6613.0,0.192594,...,2259.0,11879.0,793,11086.0,WY
3103,WY,3911.0,794.0,3013.0,0.203017,...,1136.0,4767.0,542,4225.0,WY


In [176]:
# 15 to 19 years_male_2012_y
counties_merged['FIPS Code']

0        1001.0
1        1003.0
2        1005.0
3        1007.0
4        1009.0
         ...   
3100    56037.0
3101    56039.0
3102    56041.0
3103    56043.0
3104    56045.0
Name: FIPS Code, Length: 3105, dtype: object

In [164]:
# show duplicated columns
duplicated_cols = counties_merged.columns[counties_merged.columns.duplicated()]
duplicated_cols

Index([], dtype='object')

In [161]:
# drop duplicated columns
counties_merged.drop(columns=duplicated_cols, inplace=True)

In [162]:
# save counties_merged to a geojson file
counties_merged.to_file('data/election/final_data/county_data_with_elections_2012_2016_2020_census_MAIN.geojson', driver='GeoJSON')

In [177]:
# split out the dataframes into separate dataframes for each year (2012, 2016, 2020), also take the state_abbr, county_name, state_name, and fips_code columns
counties_merged_2012 = counties_merged[[col for col in counties_merged.columns if '2012' in col] + ['state_abbr', 'county_name', 'state_name', 'FIPS Code', 'geometry']]
counties_merged_2016 = counties_merged[[col for col in counties_merged.columns if '2016' in col] + ['state_abbr', 'county_name', 'state_name', 'FIPS Code', 'geometry']]
# for 2020, include values that contain 2021 as well
counties_merged_2020 = counties_merged[[col for col in counties_merged.columns if '2020' in col or '2021' in col] + ['state_abbr', 'county_name', 'state_name', 'FIPS Code', 'geometry']]

In [201]:
# show all cols in counties_merged that have 2020 in them
[col for col in counties_merged.columns if '2016' in col]

['Civilian_labor_force_2016',
 'Employed_2016',
 'Unemployed_2016',
 'Unemployment_rate_2016',
 'votes_dem_2016',
 'votes_gop_2016',
 'total_votes_2016',
 'per_dem_2016',
 'per_gop_2016',
 'diff_2016',
 'per_point_diff_2016',
 'winner_2016',
 'Total:_2016',
 '\xa0\xa0\xa0\xa0American Indian and Alaska Native alone_2016',
 '\xa0\xa0\xa0\xa0Asian alone_2016',
 '\xa0\xa0\xa0\xa0Black or African American alone_2016',
 '\xa0\xa0\xa0\xa0Native Hawaiian and Other Pacific Islander alone_2016',
 '\xa0\xa0\xa0\xa0Some other race alone_2016',
 '\xa0\xa0\xa0\xa0Two or more races:_2016',
 '\xa0\xa0\xa0\xa0White alone_2016',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Two races excluding Some other race, and three or more races_2016',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Two races including Some other race_2016',
 'Population_16_years_and_over_Total_2016',
 'Population_16_years_and_over_labor_force_2016',
 'Population_16_years_and_over_employed_2016',
 'Population_16_years_and_over_unemployment_rate_2016',
 '\xa

In [198]:
counties_merged[[col for col in counties_merged.columns if '2020' in col] + ['state_abbr', 'county_name', 'state_name', 'FIPS Code', 'geometry']]

,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,CENSUS_2020_POP,...,state_abbr,county_name,state_name,FIPS Code,geometry
0,26425,25023,1402,5.3,58805,...,AL,Autauga County,Alabama,1001.0,"POLYGON ((-86.9176 32.66417, -86.81657 32.6601..."
1,98979,92893,6086,6.1,231767,...,AL,Baldwin County,Alabama,1003.0,"POLYGON ((-88.02927 30.22271, -88.02399 30.230..."
2,8684,8017,667,7.7,25223,...,AL,Barbour County,Alabama,1005.0,"POLYGON ((-85.74142 31.61961, -85.72983 31.632..."
3,8717,8085,632,7.3,22293,...,AL,Bibb County,Alabama,1007.0,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
4,25136,24010,1126,4.5,59134,...,AL,Blount County,Alabama,1009.0,"POLYGON ((-86.96336 33.85822, -86.9202 33.8734..."
...,...,...,...,...,...,...,...,...,...,...,...
3100,20510,18981,1529,7.5,42272,...,WY,Sweetwater County,Wyoming,56037.0,"POLYGON ((-110.05371 42.27074, -109.543 42.264..."
3101,15138,14237,901,6.0,23331,...,WY,Teton County,Wyoming,56039.0,"POLYGON ((-111.05533 44.66626, -111.00076 44.6..."
3102,9105,8525,580,6.4,20450,...,WY,Uinta County,Wyoming,56041.0,"POLYGON ((-111.04655 41.25163, -111.04655 41.2..."
3103,3944,3736,208,5.3,7685,...,WY,Washakie County,Wyoming,56043.0,"POLYGON ((-108.55056 44.16846, -108.09077 44.1..."


In [188]:
# in counties_merged_2012, fix values in all columns that contain a percentage sign in the value (i.e. some columns have values with percentage signs in them, need to be float)
for col in counties_merged_2012.columns:
    if '%' in counties_merged_2012[col].values:
        counties_merged_2012[col] = counties_merged_2012[col].str.replace('%', '').astype(float)

# in counties_merged_2016, fix values in all columns that contain a percentage sign in the value (i.e. some columns have values with percentage signs in them, need to be float)
for col in counties_merged_2016.columns:
    if '%' in counties_merged_2016[col].values:
        counties_merged_2016[col] = counties_merged_2016[col].str.replace('%', '').astype(float)

# in counties_merged_2020, fix values in all columns that contain a percentage sign in the value (i.e. some columns have values with percentage signs in them, need to be float)
for col in counties_merged_2020.columns:
    if '%' in counties_merged_2020[col].values:
        counties_merged_2020[col] = counties_merged_2020[col].str.replace('%', '').astype(float)

C:\Users\bovam\AppData\Local\Temp\ipykernel_21892\689539826.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if '%' in counties_merged_2012[col].values:
C:\Users\bovam\AppData\Local\Temp\ipykernel_21892\689539826.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if '%' in counties_merged_2016[col].values:
C:\Users\bovam\AppData\Local\Temp\ipykernel_21892\689539826.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if '%' in counties_merged_2020[col].values:


In [189]:
counties_merged_2020.per_point_diff_2020

0       0.444184
1       0.537623
2       0.076631
3       0.577280
4       0.800022
          ...   
3100    0.506294
3101   -0.375213
3102    0.628058
3103    0.646560
3104    0.775551
Name: per_point_diff_2020, Length: 3105, dtype: float64

In [190]:
# save merged dataframes to geojson files
counties_merged_2012.to_file('data/election/final_data/county_data_with_elections_2012_census_MAIN.geojson', driver='GeoJSON')
counties_merged_2016.to_file('data/election/final_data/county_data_with_elections_2016_census_MAIN.geojson', driver='GeoJSON')
counties_merged_2020.to_file('data/election/final_data/county_data_with_elections_2020_census_MAIN.geojson', driver='GeoJSON')

In [193]:
counties_merged_2020

,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,...,state_abbr,county_name,state_name,FIPS Code,geometry
0,26425,25023,1402,5.3,26545,...,AL,Autauga County,Alabama,1001.0,"POLYGON ((-86.9176 32.66417, -86.81657 32.6601..."
1,98979,92893,6086,6.1,99953,...,AL,Baldwin County,Alabama,1003.0,"POLYGON ((-88.02927 30.22271, -88.02399 30.230..."
2,8684,8017,667,7.7,8280,...,AL,Barbour County,Alabama,1005.0,"POLYGON ((-85.74142 31.61961, -85.72983 31.632..."
3,8717,8085,632,7.3,8641,...,AL,Bibb County,Alabama,1007.0,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
4,25136,24010,1126,4.5,25377,...,AL,Blount County,Alabama,1009.0,"POLYGON ((-86.96336 33.85822, -86.9202 33.8734..."
...,...,...,...,...,...,...,...,...,...,...,...
3100,20510,18981,1529,7.5,19354,...,WY,Sweetwater County,Wyoming,56037.0,"POLYGON ((-110.05371 42.27074, -109.543 42.264..."
3101,15138,14237,901,6.0,15906,...,WY,Teton County,Wyoming,56039.0,"POLYGON ((-111.05533 44.66626, -111.00076 44.6..."
3102,9105,8525,580,6.4,8812,...,WY,Uinta County,Wyoming,56041.0,"POLYGON ((-111.04655 41.25163, -111.04655 41.2..."
3103,3944,3736,208,5.3,3939,...,WY,Washakie County,Wyoming,56043.0,"POLYGON ((-108.55056 44.16846, -108.09077 44.1..."


In [143]:
# save the reshaped data to csvs
df_race_2012_reshaped.to_csv('data/election/census/census_race_2012_reshaped.csv', index=False)
df_race_2016_reshaped.to_csv('data/election/census/census_race_2016_reshaped.csv', index=False)
df_race_2020_reshaped.to_csv('data/election/census/census_race_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_age_sex_2012_reshaped.to_csv('data/election/census/census_age_sex_2012_reshaped.csv', index=False)
df_age_sex_2016_reshaped.to_csv('data/election/census/census_age_sex_2016_reshaped.csv', index=False)
df_age_sex_2020_reshaped.to_csv('data/election/census/census_age_sex_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_age_employment_2012_reshaped.to_csv('data/election/census/census_age_employment_2012_reshaped.csv', index=False)
df_age_employment_2016_reshaped.to_csv('data/election/census/census_age_employment_2016_reshaped.csv', index=False)
df_age_employment_2020_reshaped.to_csv('data/election/census/census_age_employment_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_industry_2012_reshaped.to_csv('data/election/census/census_industry_2012_reshaped.csv', index=False)
df_industry_2016_reshaped.to_csv('data/election/census/census_industry_2016_reshaped.csv', index=False)
df_industry_2020_reshaped.to_csv('data/election/census/census_industry_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_housing_financial_2012_reshaped.to_csv('data/election/census/census_housing_financial_2012_reshaped.csv', index=False)
df_housing_financial_2016_reshaped.to_csv('data/election/census/census_housing_financial_2016_reshaped.csv', index=False)
df_housing_financial_2020_reshaped.to_csv('data/election/census/census_housing_financial_2020_reshaped.csv', index=False)
# save the reshaped data to csvs
df_veteran_status_2012_reshaped.to_csv('data/election/census/census_veteran_status_2012_reshaped.csv', index=False)
df_veteran_status_2016_reshaped.to_csv('data/election/census/census_veteran_status_2016_reshaped.csv', index=False)
df_veteran_status_2020_reshaped.to_csv('data/election/census/census_veteran_status_2020_reshaped.csv', index=False)

##### Other stuff

In [ ]:
# add _2012 to all column names, remove ':' from column names
df2_fips.columns = [col + '_2012' if col != 'state_name' and col != 'county_name' else col for col in df2_fips.columns]
df2_fips.columns = [col.replace(':', '') for col in df2_fips.columns]
df2_fips

In [ ]:
# remove commas from all values
df2_fips = df2_fips.replace(',', '', regex=True)
df2_fips

In [ ]:
import pandas as pd

def add_demographic_percentages(df):
    """
    Add percentage columns for each demographic category.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with demographic counts
    
    Returns:
    pandas.DataFrame: DataFrame with additional percentage columns
    """
    result = df.copy()
    
    # Define the demographic columns and their new percentage column names
    demographic_cols = {
        'American Indian and Alaska Native alone_2012': 'pct_aian_2012',
        'Asian alone_2012': 'pct_asian_2012',
        'Black or African American alone_2012': 'pct_black_2012',
        'Native Hawaiian and Other Pacific Islander alone_2012': 'pct_nhpi_2012',
        'Some other race alone_2012': 'pct_other_2012',
        'Two or more races_2012': 'pct_two_or_more_2012',
        'White alone_2012': 'pct_white_2012',
        'Two races excluding Some other race, and three or more races_2012': 'pct_two_races_excl_other_2012',
        'Two races including Some other race_2012': 'pct_two_races_incl_other_2012'
    }
    
    # Calculate percentages for each demographic category
    for col, new_col in demographic_cols.items():
        result[new_col] = (result[col] / result['Total_2012'] * 100).round(2)
        
    # Validate that percentages sum to approximately 100
    # Note: We exclude the detailed two races breakdowns as they're subcategories
    main_pct_cols = [
        'pct_aian_2012',
        'pct_asian_2012',
        'pct_black_2012',
        'pct_nhpi_2012',
        'pct_other_2012',
        'pct_two_or_more_2012',
        'pct_white_2012'
    ]
    
    # Calculate sum of percentages
    result['pct_sum'] = result[main_pct_cols].sum(axis=1)
    
    # Check for any significant deviations from 100%
    tolerance = 0.1  # Allow for small rounding differences
    deviations = result[abs(result['pct_sum'] - 100) > tolerance]
    if len(deviations) > 0:
        print(f"\nWarning: {len(deviations)} rows have percentages that don't sum to 100% (±{tolerance}%):")
        for _, row in deviations.iterrows():
            print(f"- {row['county_name']}, {row['state_name']}: {row['pct_sum']:.2f}%")
    
    # Drop the validation column
    result = result.drop('pct_sum', axis=1)
    
    # Group and order columns
    id_cols = ['county_name', 'state_name', 'fips_code_2012', 'state_abbr_2012']
    count_cols = ['Total_2012'] + list(demographic_cols.keys())
    pct_cols = list(demographic_cols.values())
    
    # Reorder columns
    result = result[id_cols + count_cols + pct_cols]
    
    return result

In [ ]:
# Assuming your dataframe is called df
df2_fips = add_demographic_percentages(df2_fips)